# Core Problem
### Identify duplicate or matching individuals across CSVs with messy data

In [1]:
import pandas as pd
from rapidfuzz import fuzz
import string

df1 = pd.read_csv("general.csv")
df2 = pd.read_csv("political.csv")
schema = pd.read_csv('colsofinterest.csv')


In [2]:
#below we will select the cols of interest
#Below is all the col we will match on
col_list = []
for i,e in schema.values:
    col_list.append(i)

#below is all the cols that are string so we can format them
string_cols = []
cond = schema['data_type'] == 'string'
for i,e in schema[cond].values:
    string_cols.append(i)



In [3]:
#Data cleaning
#taking all string columns and stripping and lowering them
for col in string_cols:
    df1[col] = df1[col].str.lower().str.strip()

for col in string_cols:
    df2[col] = df2[col].str.lower().str.strip()

In [4]:
#Create a new index type column so we can track its original location
def create_ind(df,tb_name):
    df['og_loc'] = df.index
    df['source'] = tb_name
    return df
    
col_list.append('og_loc')    
col_list.append('source')    
df2 = create_ind(df2,'df2')
df1 = create_ind(df1,'df1')

In [5]:
#combining both df to be able to get our scores
comb_df = pd.concat([df2[col_list],df1[col_list]], axis = 0)

In [6]:
# Heuristic-based duplicate detection (blocking)

# NOTE: loop-based implementation for clarity.
# will vectorized later

comb_df['block_key_1'] = ''  # start with empty strings
comb_df['block_key_2'] = ''  # start with empty strings
comb_df['block_key_3'] = ''  # start with empty strings

#removing below since we do not need them for the bottom steps will readd 
col_list.remove('og_loc')    
col_list.remove('source')  

for col in col_list:
    comb_df['block_key_1'] += comb_df[col].str[:2].fillna('')

for col in col_list:
    comb_df['block_key_2'] += comb_df[col].str[:3].fillna('')

for col in col_list:
    comb_df['block_key_3'] += comb_df[col].str[:4].fillna('')

#adding them back in since we will use them to keep these cols
col_list.append('og_loc')    
col_list.append('source')   

In [7]:
#Creating the dup flag
# doing keep = false to mark all duplciates as 1
comb_df['dup_flag1'] = comb_df['block_key_1'].duplicated(keep=False).astype(int)
comb_df['dup_flag2'] = comb_df['block_key_2'].duplicated(keep=False).astype(int)
comb_df['dup_flag3'] = comb_df['block_key_3'].duplicated(keep=False).astype(int)
comb_df = comb_df.sort_values(by=['dup_flag3','dup_flag2','dup_flag1'], ascending = False)


In [8]:
def get_score(row):
    if row['dup_flag1']:
        if row['dup_flag2']:
            if row['dup_flag3']:
                return 100
            return 66
        return 33
    return 0

comb_df['heu'] = comb_df.apply(get_score, axis = 1)
col_list.extend(['block_key_1','block_key_2','block_key_3','heu'])
comb_df = comb_df[col_list]
col_list.remove('heu')
col_list.remove('og_loc')    
col_list.remove('source')  

In [9]:
group_info = (comb_df.groupby('block_key_3').agg(n_rows = ('og_loc','count'),
                n_sources = ('source','nunique')))
#Because every row has an og_loc, counting og_loc is basically counting rows.
    #n_rows = 1 → the signature appears once (no duplicate candidate)
    #n_rows = 2 → exactly two candidate records share the signature
    #n_rows > 2 → multiple records share signature (ambiguous cluster)


#n_sources = how many distinct sources exist inside the bucket
#Since source is df1 or df2, nunique() tells you whether the bucket contains:
    #only df1
    #only df2
    #or both
#Interpretation:
    #n_sources == 1 → all matching records came from one table
    #✅ same-table duplicate (internal duplicate)
#n_sources == 2 → bucket includes df1 and df2
    #✅ cross-table match

#wE ARE ONLY DOING THIS FOR BLOCK KEY 3 SINCE THESE ARE HIGH CONFIDENCE 

In [10]:
comb_df.head(1)

,fname,lname,email,og_loc,source,block_key_1,block_key_2,block_key_3,heu
3,sara,williams,sarah.williams@outlook.com,3,df2,sawisa,sarwilsar,sarawillsara,100


In [11]:
#Splitting back into the original dfs so we can do fuzzy matching
df1 = comb_df[comb_df['source'] == 'df1'].copy()
df2 = comb_df[comb_df['source'] == 'df2'].copy()

#Now breaking into pairs so we do not wast compute doing fuzzy matching on all rows
pairs_1 = df1.merge(df2, on="block_key_1", how="inner")
pairs_2 = df1.merge(df2, on="block_key_2", how="inner")
pairs_3 = df1.merge(df2, on="block_key_3", how="inner")

candidates = pd.concat([pairs_1, pairs_2, pairs_3]).drop_duplicates()
candidates = candidates.rename(columns={
    "fname_x": "fname_1",
    "lname_x": "lname_1",
    "dob_x": "dob_1",
    "email_x": "email_1",
    "fname_y": "fname_2",
    "lname_y": "lname_2",
    "dob_y": "dob_2",
    "email_y": "email_2",
})
    

In [12]:
#looping through the candidates to do fuzzy matching 

#cleaning up col_list so we can iterate through it in fuzzy match
col_list.remove('block_key_1')
col_list.remove('block_key_2')
col_list.remove('block_key_3')

col_list



for col in col_list:

    scores = []

    for _, row in candidates.iterrows():
        v1 = row[f"{col}_1"]
        v2 = row[f"{col}_2"]

        if pd.isna(v1) or pd.isna(v2):
            scores.append(0)
        else:
            scores.append(fuzz.partial_ratio(str(v1), str(v2)))

    candidates[f"{col}_score"] = scores



In [14]:
#combining the fuzzy scores


score_cols = candidates.filter(like="_score").columns
candidates["final_score_fuzzy"] = candidates[score_cols].mean(axis=1)
candidates["final_score_fuzzy"] = (candidates["final_score_fuzzy"] + (candidates["heu_x"]+candidates["heu_y"])/2)/2


In [16]:
cond = candidates['final_score_fuzzy'] < 80
candidates[cond]

,fname_1,lname_1,email_1,og_loc_x,source_x,block_key_1,block_key_2_x,block_key_3_x,heu_x,fname_2,...,block_key_3_y,heu_y,block_key_1_x,block_key_2,block_key_1_y,block_key_3,fname_score,lname_score,email_score,final_score_fuzzy
16,john,smith,john.smith@gmail.com,0,df1,josmjo,johsmijoh,johnsmitjohn,33,jon,...,jonsmitjohn,33,NaN,NaN,NaN,NaN,80.000000,100.0,100.0,63.166667
17,mike,johnson,mike.j@gmail.com,2,df1,mijomi,mikjohmik,mikejohnmike,33,michael,...,michjohnmike,33,NaN,NaN,NaN,NaN,66.666667,100.0,100.0,60.944444
